# Neural Network function

在这里我们先了解一下基本的神经网络函数.

我们先从计算机视觉方(compute version)面入手,结合多方面的资料来共同理解.

### 1 Binary Classification

如下图是一张猫的图片,每一个像素点是一个(RGB)三维的向量,如果我们要判断这张图是不是猫.也就是说输出非0即1.

![](../picture/01.png)

像此类问题就是二分类的问题,基本上其标签是属于伯努利分布.

那么对于像图片我们可以将所有的像素点转换成一个列向量,列向量中包含RGB所有的值.比如这张图片(64,64,3)转换后的形式为:

$x^{(1)}=\begin{bmatrix}
255\\
236 \\
...\\ 
0
\end{bmatrix}_{(64*64*3,1)}$



**Ps:**

- 图片(RGB),三个值代表红色,绿色,蓝色.
- 数组(64,64,3):表示长64,宽64,通道3.
- 如果不清楚图片的存储模式可以使用Python代码查看图片转换成数组后的形式:
```python
import matplotlib.pyplot as plt
plt.imread('path...')```

那么我们可以有如下定义符号.

用一对$(x,y)$来表示一个单独的样本,其中我们定义$n_x$为样本的特征(也就是一张图片的所有像素点($64*64*3$),fetures),定义$y$为样本标签$labels$.$x\in R^{n_x},y\in \{0,1\}$

那么对于$m$和样本而言其样本的形式为:

$X=[x^{(1)},x^{(2)},x^{(3)},...,x^{(m)}]$ 形状(shape): (n_x,m)

$Y=[y^{(1)},y^{(2)},y^{(3)},...,y^{(m)}]$ 形状(shape): (1,m)

**Ps:**

如果一个向量中含有多个特征,我们一般也会称之为**张量(Tensor)**

那么我们现在已经知道了一张图片如何用数组(向量)表示.但是如果现在给予一张同样大小的图片,我们还是不知道该怎样计算去预测这张图片是不是猫.

### 2 Logistic Regression

如果对LR还不是很了解的朋友建议要先看[Classic_machine_learning](https://github.com/woaij100/Classic_machine_learning)中的LR.否则下面的内容可能无法了解.

我们可以使用LR来进行预测.比如如果这张图片是猫,那么我们将labels=1,否则lables=0.也就是说:

$Given\;x,want\; \hat{y},actually,\hat{y}= P(y = 1 | x)$

其中$\hat{y}$表示我们的预测值.

那么对于LR:

在训练样本给定的下,我们需要使用参数$W,b$来拟合模型,即:

Parameters: $W \in R^{n_x},b \in R$, W:Weights,b:Bias.

Output: $\hat{Y}=sigmoid(W^{T}X+b)$.

我们可以定义成本函数:成本函数,可以用来衡量全体训练样本上的表现.在训练的过程中成本函数会逐渐降低.

cost function:$J(\hat{Y},Y) =\frac{1}{m}\cdot \sum_{i=1}^{m}(- (y_i*log(\hat{Y_i}) + (1-Y_i)*log(1-\hat{Y_i})))$



**Ps:**

- 在Sigmoid函数中,我们到底要不要使用转置是相对于你初始化$W$的形状而言的,最终无论如何只要$\hat{Y}$能够与$Y$是一样的就可以.


- 实际上LR就是一个小型的神经网络(后面就会逐渐理解)


现在有了损失函数的定义,初始化参数$W,b$.那么我们现在需要使用梯度下降法来更新参数

Recap:

$\hat{y} = sigmoid(w^{T}x + b),where sigmoid(z) = \frac{1}{1+e^{-z}}$

$J(w,b) =  - (1/m) * \sum_{i=1}^{m}[(y_i*log(\hat{y_i}) + (1-y_i)*log(1-\hat{y_i}))]$

于是想要找到最优的$W,b$使得minimize J(w,b).

根据极小损失函数(经验风险最小化)使用链式法则得到:

$W := W - \alpha dW$

$b := b - \alpha db$

其中$\alpha$表示学习率.

梯度下降法就是从初始化的点朝最陡的下坡方向走一步,也就是下降方向最快的方向走一步,通过多次迭代,会逐渐找到最优的点,也就是全局收敛的最优解.

[PDF](../PDFS/C1W2L04%slides.pdf)

### 3. Vectorizing Logistic Regression

由于Python是比较低效的语言,所以我们如果用多重for循环来计算LR,会发现代码的执行效率非常低.所以我们需要使用向量的方式(Numpy)来加速计算的速度.

首先我们定义一个$X$矩阵$X = [x^{(1)},x^{(2)},x^{(3)},x^{(m)}]$ shape is (n_x,m), 

$Z = np.dot(w.T,X) + b  (broadcasting)$

$A = [a^{(1)},a^{(2)},a^{(3)},....,a^{(m)}] = sigmoid(Z)$

**Ps:**
这里的$b$能和一个向量相加,是由于Numpy中存在𝑏𝑟𝑜𝑎𝑑𝑐𝑎𝑠𝑡𝑖𝑛𝑔也就是广播机制.

那么对于参宿更新也同样使用向量的形式:

$dZ= A-Y$

$db =\frac{1}{m} \cdot np.sum(dZ)$

$dw = \frac{1}{m} \cdot np.dot(X,dZ^T)$

那么最后的代码形式如下:

```python
Z = np.dot(W.T,X) + b
A = sigmoid(Z)
cost = np.sum(-(np.multiply(Y,np.log(A))+np.multiply((1-Y),np.log(1-A))))/m

dZ = A-Y
dW = np.dot(dZ.T,X)/m
db = np.sum(dZ,keepdims=True,axis=1)```

**Ps:**

在numpy中如果要创建一个mx1,或者1xm 的矩阵，size或者shape一定要写成(m,1) 的形式，否则返回出的shape是(m,) 而不是(m,1)，那么在进行矩阵运算的时候将会出现一些错误

[PDF](../PDFS/C1W2L16%slides.pdf)